<a href="https://colab.research.google.com/github/francianerod/PROJETOS-Empresas/blob/main/Louzan_fase1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Relatórios Gerenciais - Resultados 1**

Analista responsável: Franciane Rodrigues

Empresa: Louzan Illustrations

Área/Setor: Administrativo/Estratégico

Obs.: Código usado para confecção de relatórios gerenciais das coleções de camisetas pretas

## **Instalação e Preparação de Ambiente**

In [ ]:
# Comando de abertura para Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Instalação de Pacotes
!pip install gspread

In [ ]:
# Importação de biblioteca
import pandas as pd
from google.colab import auth
from google.auth import default
import gspread

In [ ]:
# Autenticação
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

In [ ]:
# Configuração da quantidade de colunas para aparecer em um Datraframe
pd.set_option('display.max_columns',100)

In [ ]:
# Ignorar avisos
import warnings
warnings.filterwarnings("ignore")

## **EXTRAÇÃO**

In [ ]:
# Abertura de Biblioteca - Usando apenas o filtro "Pagos"
df = pd.read_csv('/content/drive/MyDrive/Dados/Temporário/Agosto_2023/relatorio_pedidos_produtos_cyber_agosto.csv',
                 encoding='ISO-8859-1',
                 sep = ';')

In [ ]:
# Visualizar as últimas linhas do dataframe
df.tail(10)

## **TRANSFORMAÇÃO**

### **Transformação: Geral**

In [ ]:
# Filtragem por linhas
geral_venda = df.iloc[47:55]

# Filtragem por colunas "Fornecedor", "Vendas" e "Custo" usando .loc[]
geral_venda = geral_venda.loc[:, ["Fornecedor", "Vendas", "Custo"]]

# Renomeando colunas
geral_venda.rename(columns={'Fornecedor':'Demonstrativo',
                            'Vendas':'Produtos',
                            'Custo':'Valor ($)',
                            },inplace=True)

# Mostrar resultados gerais
geral_venda

In [ ]:
# Eliminando as linhas do dataframe que representa o resultado final
df = df.drop(range(47, 55))

In [ ]:
# Eliminando colunas desnecessárias
df.drop(['SKU', 'SKU Fornecedor', 'Fornecedor', 'Custo'],axis=1, inplace=True)

In [ ]:
# Preenchendo os valores nulos com uma string.
df['Marca'].fillna('SEM INFORMAÇÃO', inplace=True)

In [ ]:
# Trabalhar com valor monetario
df['Valor_TOTAL (R$)'] = df['Valor Vendas'].str.replace('R\$', '').str.replace(',00', '').str.replace('.', '')

# Converter a coluna para o tipo de dado 'int'
df['Valor_TOTAL (R$)'] = df['Valor_TOTAL (R$)'].astype(int)

df.head()

In [ ]:
# Converter a coluna para o tipo de dado 'int'
df['Vendas'] = df['Vendas'].astype(int)
df['Valor_TOTAL (R$)'] = df['Valor_TOTAL (R$)'].astype(int)

In [ ]:
# Crie a nova coluna 'situação' com base na coluna 'Marca'.
df['Situação'] = df['Marca'].apply(lambda x: 'EM PRODUÇÃO' if 'CYBER PUNK' in x else 'EM ESTOQUE')

In [ ]:
# Verificação
df.sample(10)

In [ ]:
# Particularidades da base de dados: Crie a nova coluna 'Situação' com base na coluna 'Marca'.
#df['Situação'] = df.apply(lambda row: 'EM ESTOQUE' if 'Dragon Ball Clássico' in row['Marca'] else 'EM PRODUÇÃO', axis=1)

In [ ]:
# Particularidades da base de dados: Substitua os valores na coluna 'situação' com base na coluna 'marca'
#marcas_substituir = ['Digimonster', 'POKEMON', 'Pokémonster', 'SEM INFORMAÇÃO', 'YU GI OH']
#df.loc[df['Marca'].isin(marcas_substituir), 'Situação'] = 'EM ESTOQUE'

### **Transformação: PRE-VENDA**

In [ ]:
# Localizando as linhas onde a coluna "marca" contém a palavra "EM PRODUÇÃO"
geral_produtos_pre = df[df['Situação'].str.contains('EM PRODUÇÃO', case=False)]

In [ ]:
# Converter a coluna para o tipo de dado 'int'
geral_produtos_pre['Vendas'] = geral_produtos_pre['Vendas'].astype(int)

# Quantidade de Camisetas da pré-venda
geral_produtos_pre.groupby('Categoria')['Vendas'].sum().sort_values(ascending=True)

In [ ]:
# Verificação
geral_produtos_pre

In [ ]:
# Valor total da pré-venda
total_pre_venda = geral_produtos_pre['Valor_TOTAL (R$)'].sum()

# Imprimir os totais
print("O valor total de vendas na Pré-venda foi de: R$", total_pre_venda)

In [ ]:
# Filtrando as linhas que começam com "PRÉ VENDA" na coluna "Produto"
geral_pre_venda = df[df['Produto'].str.startswith('PRÉ VENDA')]

In [ ]:
geral_pre_venda

In [ ]:
sorted(pd.unique(geral_pre_venda['Combinação']))

In [ ]:
# Criando um dicionário para mapear as strings para os tamanhos correspondentes.
tamanho_mapping = {'Tamanho: P': 'P',
                   'Tamanho: M': 'M',
                   'Tamanho: G': 'G',
                   'Tamanho: GG': 'GG',
                   'Tamanho: G1': 'G1',
                   'Tamanho: G2': 'G2',
                   'Tamanho: G3': 'G3'
                   }

# Criando uma função para mapear as strings usando o dicionário.
def mapear_tamanho(valor):
    return tamanho_mapping.get(valor, 'Não se aplica')

# Aplique a função mapear_tamanho para criar a nova coluna 'Tamanho'.
geral_pre_venda['Tamanho'] = geral_pre_venda['Combinação'].apply(mapear_tamanho)

In [ ]:
# Eliminando a coluna Combinação
geral_pre_venda.drop(['Combinação'],axis=1,inplace=True)

In [ ]:
geral_pre_venda

In [ ]:
# Calcule a soma da coluna de vendas
total_vendas = geral_pre_venda['Vendas'].sum()

# Exiba o total de vendas
print(f'Soma do total de Camisetas da Pré-Venda: {total_vendas}')

In [ ]:
# Converter a coluna para o tipo de dado 'int'
geral_pre_venda['Vendas'] = geral_pre_venda['Vendas'].astype(int)

# Defina a ordem categórica desejada
ordem_categorica = ['P', 'M', 'G', 'GG', 'G1', 'G2', 'G3']

# Use a função Categorical para definir a ordem
geral_pre_venda['Tamanho'] = pd.Categorical(geral_pre_venda['Tamanho'], categories=ordem_categorica, ordered=True)

# Calcule a soma das vendas por Tamanho
geral_grade = geral_pre_venda.groupby('Tamanho')['Vendas'].sum()

# Reordene o resultado com base na ordem categórica
geral_grade = geral_grade.reindex(ordem_categorica)

# Crie um DataFrame a partir do resultado
geral_grade = pd.DataFrame({'Tamanho': geral_grade.index, 'Quantidade': geral_grade.values})

# Exiba a tabela resultante
geral_grade

In [ ]:
sorted(pd.unique(geral_pre_venda['Produto']))

In [ ]:
# Localizando as linhas onde a coluna "Produto" e realizando separação
pre_venda_p1 = geral_pre_venda[geral_pre_venda['Produto'].str.contains('PRÉ VENDA - DAVID MARTINEZ', case=False)]

# Converter a coluna para o tipo de dado 'int'
pre_venda_p1['Tamanho'] = pre_venda_p1['Tamanho'].astype(str)

# Defina a ordem categórica desejada
ordem_categorica = ['P', 'M', 'G', 'GG', 'G1', 'G2', 'G3']

# Use a função Categorical para definir a ordem
pre_venda_p1['Tamanho'] = pd.Categorical(pre_venda_p1['Tamanho'], categories=ordem_categorica, ordered=True)

# Ordene o DataFrame com base na coluna 'Tamanho'
pre_venda_p1 = pre_venda_p1.sort_values(by='Tamanho')

# Eliminando colunas
pre_venda_p1.drop(['Valor Vendas', 'Situação', 'Valor_TOTAL (R$)'],axis=1,inplace=True)

# Exiba o DataFrame resultante
pre_venda_p1

In [ ]:
# Separando as linhas da coluna "Produto"
pre_venda_p2 = geral_pre_venda[geral_pre_venda['Produto'].str.contains('PRÉ VENDA - LUCY', case=False)]

# Converter a coluna para o tipo de dado 'int'
pre_venda_p2['Tamanho'] = pre_venda_p2['Tamanho'].astype(str)

# Defina a ordem categórica desejada
ordem_categorica = ['P', 'M', 'G', 'GG', 'G1', 'G2', 'G3']

# Use a função Categorical para definir a ordem
pre_venda_p2['Tamanho'] = pd.Categorical(pre_venda_p2['Tamanho'], categories=ordem_categorica, ordered=True)

# Ordene o DataFrame com base na coluna 'Tamanho'
pre_venda_p2 = pre_venda_p2.sort_values(by='Tamanho')

# Eliminando colunas
pre_venda_p2.drop(['Valor Vendas', 'Situação', 'Valor_TOTAL (R$)'],axis=1,inplace=True)

# Exiba o DataFrame resultante
pre_venda_p2

In [ ]:
# Separando as linhas da coluna "Produto"
pre_venda_p3 = geral_pre_venda[geral_pre_venda['Produto'].str.contains('PRÉ VENDA - REBECCA', case=False)]

# Converter a coluna para o tipo de dado 'int'
pre_venda_p3['Tamanho'] = pre_venda_p3['Tamanho'].astype(str)

# Defina a ordem categórica desejada
ordem_categorica = ['P', 'M', 'G', 'GG', 'G1', 'G2', 'G3']

# Use a função Categorical para definir a ordem
pre_venda_p3['Tamanho'] = pd.Categorical(pre_venda_p3['Tamanho'], categories=ordem_categorica, ordered=True)

# Ordene o DataFrame com base na coluna 'Tamanho'
pre_venda_p3 = pre_venda_p3.sort_values(by='Tamanho')

# Eliminando colunas
pre_venda_p3.drop(['Valor Vendas', 'Situação', 'Valor_TOTAL (R$)'],axis=1,inplace=True)

# Exiba o DataFrame resultante
pre_venda_p3

In [ ]:
"""
# Separando as linhas da coluna "Produto"
pre_venda_p4 = geral_pre_venda[geral_pre_venda['Produto'].str.contains('PRÉ VENDA - CAMISETA VEGETA E NAPPA', case=False)]

# Converter a coluna para o tipo de dado 'int'
pre_venda_p4['Tamanho'] = pre_venda_p4['Tamanho'].astype(str)

# Defina a ordem categórica desejada
ordem_categorica = ['P', 'M', 'G', 'GG', 'G1', 'G2', 'G3']

# Use a função Categorical para definir a ordem
pre_venda_p4['Tamanho'] = pd.Categorical(pre_venda_p4['Tamanho'], categories=ordem_categorica, ordered=True)

# Ordene o DataFrame com base na coluna 'Tamanho'
pre_venda_p4 = pre_venda_p4.sort_values(by='Tamanho')

# Eliminando colunas
pre_venda_p4.drop(['Valor Vendas', 'Situação', 'Valor_TOTAL (R$)'],axis=1,inplace=True)

# Exiba o DataFrame resultante
pre_venda_p4
"""

### **Transformação: PRONTA-ENTREGA**

In [ ]:
# Localizando as linhas onde a coluna "marca" contém a palavra "EM ESTOQUE"
estoque_venda = df[df['Situação'].str.contains('EM ESTOQUE', case=False)]

In [ ]:
estoque_venda

In [ ]:
# Criando um dicionário para mapear as strings para os tamanhos correspondentes.
tamanho_mapping = {'Tamanho: P': 'P',
                   'Tamanho: M': 'M',
                   'Tamanho: G': 'G',
                   'Tamanho: GG': 'GG',
                   'Tamanho: G1': 'G1',
                   'Tamanho: G2': 'G2',
                   'Tamanho: G3': 'G3'
                   }

# Criando uma função para mapear as strings usando o dicionário.
def mapear_tamanho(valor):
    return tamanho_mapping.get(valor, 'Não se aplica')

# Aplique a função mapear_tamanho para criar a nova coluna 'Tamanho'.
estoque_venda['Tamanho'] = estoque_venda['Combinação'].apply(mapear_tamanho)

In [ ]:
estoque_venda

In [ ]:
# Converter a coluna para o tipo de dado 'int'
estoque_venda['Tamanho'] = estoque_venda['Tamanho'].astype(str)

# Defina a ordem categórica desejada
ordem_categorica = ['P', 'M', 'G', 'GG', 'G1', 'G2', 'G3', 'Não se aplica']

# Use a função Categorical para definir a ordem
estoque_venda['Tamanho'] = pd.Categorical(estoque_venda['Tamanho'], categories=ordem_categorica, ordered=True)

# Ordene o DataFrame com base na coluna 'Tamanho'
estoque_venda = estoque_venda.sort_values(by='Tamanho')

# Eliminando colunas
estoque_venda.drop(['Combinação'],axis=1,inplace=True)

# Exiba o DataFrame resultante
estoque_venda

In [ ]:
# Converter a coluna para o tipo de dado 'int'
estoque_venda['Vendas'] = estoque_venda['Vendas'].astype(int)

# Quantidade de Camisetas da pré-venda por Tamanho
estoque_venda.groupby('Categoria')['Vendas'].sum().sort_values(ascending=True)

In [ ]:
# Valor total da pré-venda
total_estoque = estoque_venda['Valor_TOTAL (R$)'].sum()

# Imprimir os totais
print("O valor total de vendas a pronto-entrega durante a Pré-venda foi de: R$", total_estoque)

## **RESULTADOS GERAIS**

### **RELATÓRIO FINANCEIRO**

In [ ]:
# Demonstrativo Financeiro
geral_venda

### **RELATÓRIO COMERCIAL**

In [ ]:
# Quantidade de produtos vendidos apenas na pré-venda
geral_produtos_pre.groupby('Categoria')['Vendas'].sum().sort_values(ascending=True)

In [ ]:
# Preferência do público por personagem na pré-venda
geral_produtos_pre.groupby('Produto')['Vendas'].sum().sort_values(ascending=False)

In [ ]:
# Valor total da pré-venda
total_pre_venda = geral_produtos_pre['Valor_TOTAL (R$)'].sum()

# Imprimir os totais
print("O valor total de vendas na Pré-venda foi de: R$", total_pre_venda)

### **RELATÓRIO DE INSUMO**

In [ ]:
# Solicitação para camiseteria
geral_grade

### **RELATÓRIO OPERACIONAL**

In [ ]:
# Verificação de produtos que precisam ser produzidos
pd.unique(geral_produtos_pre['Categoria'])

In [ ]:
# Quantidade de produtos a serem produzidos
geral_produtos_pre.groupby('Categoria')['Vendas'].sum().sort_values(ascending=True)

In [ ]:
# Para produção
pre_venda_p1

In [ ]:
# Para produção
pre_venda_p2

In [ ]:
# Para produção
pre_venda_p3

In [ ]:
# Para produção
#pre_venda_p4

### **RELATÓRIO PRONTO-ENTREGA**

In [ ]:
# Verificação de produtos que foram vendidos
pd.unique(estoque_venda['Categoria'])

In [ ]:
# Valor total da pré-venda
total_estoque = estoque_venda['Valor_TOTAL (R$)'].sum()

# Imprimir os totais
print("O valor total de vendas na Pré-venda foi de: R$", total_estoque)

In [ ]:
# Listagem total do que foi vendido
estoque_venda.drop(['Valor Vendas', 'Valor_TOTAL (R$)'],axis=1,inplace=True)

In [ ]:
estoque_venda

In [ ]:
# Separar produtos em estoque
estoque_venda_p1 = estoque_venda[estoque_venda['Categoria'] == 'CAMISETAS']
estoque_venda_p2 = estoque_venda[estoque_venda['Categoria'] == 'QUADROS']
estoque_venda_p3 = estoque_venda[estoque_venda['Categoria'] == 'STICKERS']
#estoque_venda_p4 = estoque_venda[estoque_venda['Categoria'] == 'OUTLET']

In [ ]:
# Mostrar as tabelas separadas
estoque_venda_p1

In [ ]:
# Mostrar as tabelas separadas
estoque_venda_p2

In [ ]:
# Mostrar as tabelas separadas
estoque_venda_p3

In [ ]:
# Mostrar as tabelas separadas
#estoque_venda_p4

## **CARREGAMENTO**

Esquemas de tabelas históricas

*   geral_venda - Resultado financeiro
*   geral_produtos_pre - todos os produtos da coleção vigente;
*   geral_pre_venda - só camisetas da coleção vigente;
*   geral_grade - só camisetas da coleção vigente
*   geral_grade_cores - grade completa com cores e tamanhos (se houver)
*   branco_grade_cores - grade por cor - (se houver)
*   preto_grade_cores - grade por cor - (se houve)
*   pre_venda_p1 - qtd. de produção do personagem 1
*   pre_venda_p2 - qtd. de produção do personagem 2
*   pre_venda_p3 - qtd. de produção do personagem 3
*   pre_venda_p4 - qtd. de produção do personagem 4
*   estoque_venda - Tudo que foi vendido a pronto-entrega durante o mês vigente
*   estoque_venda_p1 - camisetas a pronto-entrega
*   estoque_venda_p2 - quadros a pronto-entrega
*   estoque_venda_p3 - stickers a pronto-entrega
*   estoque_venda_p4 - outlet a pronto-entrega

In [ ]:
# Carregamento em uma pasta Google Drive em extensão .csv
geral_venda.to_csv('/content/drive/MyDrive/Dados/Tratado/Agosto_2023/Agosto_2023_Dados Históricos/agosto_2023_geral_venda.csv')
geral_produtos_pre.to_csv('/content/drive/MyDrive/Dados/Tratado/Agosto_2023/Agosto_2023_Dados Históricos/agosto_2023_geral_produtos_pre.csv')

geral_pre_venda.to_csv('/content/drive/MyDrive/Dados/Tratado/Agosto_2023/Agosto_2023_Dados Históricos/agosto_2023_geral_pre_camisetas')
geral_grade.to_csv('/content/drive/MyDrive/Dados/Tratado/Agosto_2023/Agosto_2023_Dados Históricos/agosto_2023_grade_geral.csv')

pre_venda_p1.to_csv('/content/drive/MyDrive/Dados/Tratado/Agosto_2023/Agosto_2023_Dados Históricos/agosto_2023_pre_venda_p1.csv')
pre_venda_p2.to_csv('/content/drive/MyDrive/Dados/Tratado/Agosto_2023/Agosto_2023_Dados Históricos/agosto_2023_pre_venda_p2.csv')
pre_venda_p3.to_csv('/content/drive/MyDrive/Dados/Tratado/Agosto_2023/Agosto_2023_Dados Históricos/agosto_2023_pre_venda_p3.csv')
#pre_venda_p4.to_csv('/content/drive/MyDrive/Dados/Tratado/Agosto_2023/Agosto_2023_Dados Históricos/agosto_2023_pre_venda_p4.csv')

estoque_venda.to_csv('/content/drive/MyDrive/Dados/Tratado/Agosto_2023/Agosto_2023_Dados Históricos/agosto_2023_estoque_venda.csv')
estoque_venda_p1.to_csv('/content/drive/MyDrive/Dados/Tratado/Agosto_2023/Agosto_2023_Dados Históricos/agosto_2023_estoque_venda_p1.csv')
estoque_venda_p2.to_csv('/content/drive/MyDrive/Dados/Tratado/Agosto_2023/Agosto_2023_Dados Históricos/agosto_2023_estoque_venda_p2.csv')
estoque_venda_p3.to_csv('/content/drive/MyDrive/Dados/Tratado/Agosto_2023/Agosto_2023_Dados Históricos/agosto_2023_estoque_venda_p3.csv')
#estoque_venda_p4.to_csv('/content/drive/MyDrive/Dados/Tratado/Agosto_2023/Agosto_2023_Dados Históricos/agosto_2023_estoque_venda_p4.csv')

In [ ]:
# Acessar planilha que vai receber os dados
planilha = gc.open('Resultado_Agosto_2023')

In [ ]:
# Acesse as guias da planilha usando o método worksheet
agosto_geral_venda = planilha.worksheet('Página1')
agosto_produtos_pre = planilha.worksheet('Página2')
agosto_geral_grade = planilha.worksheet('Página3')

p1 = planilha.worksheet('Página4')
p2 = planilha.worksheet('Página5')
p3 = planilha.worksheet('Página6')
#p4 = planilha.worksheet('Página7')

estoque_venda_p1_1 = planilha.worksheet('Página8')
estoque_venda_p2_2 = planilha.worksheet('Página9')
estoque_venda_p3_3 = planilha.worksheet('Página10')
#estoque_venda_p4_4 = planilha.worksheet('Página11')

In [ ]:
# Exportação 1
agosto_geral_venda.update([geral_venda.columns.values.tolist()] + geral_venda.values.tolist())

In [ ]:
# Exportação 2
agosto_produtos_pre.update([geral_produtos_pre.columns.values.tolist()] + geral_produtos_pre.values.tolist())

In [ ]:
# Exportação 3
agosto_geral_grade.update([geral_grade.columns.values.tolist()] + geral_grade.values.tolist())

In [ ]:
# Exportação 4
pre_venda_p1.fillna('', inplace=True)
p1.update([pre_venda_p1.columns.values.tolist()] + pre_venda_p1.values.tolist())

In [ ]:
# Exportação 5
pre_venda_p2.fillna('', inplace=True)
p2.update([pre_venda_p2.columns.values.tolist()] + pre_venda_p2.values.tolist())

In [ ]:
# Exportação 6
pre_venda_p3.fillna('', inplace=True)
p3.update([pre_venda_p3.columns.values.tolist()] + pre_venda_p3.values.tolist())

In [ ]:
# Exportação 7
#pre_venda_p4.fillna('', inplace=True)
#p4.update([pre_venda_p4.columns.values.tolist()] + pre_venda_p4.values.tolist())

In [ ]:
# Exportação 8
estoque_venda_p1.fillna('', inplace=True)
estoque_venda_p1_1.update([estoque_venda_p1.columns.values.tolist()] + estoque_venda_p1.values.tolist())

In [ ]:
# Exportação 9
estoque_venda_p2.fillna('', inplace=True)
estoque_venda_p2_2.update([estoque_venda_p2.columns.values.tolist()] + estoque_venda_p2.values.tolist())

In [ ]:
# Exportação 10
estoque_venda_p3.fillna('', inplace=True)
estoque_venda_p3_3.update([estoque_venda_p3.columns.values.tolist()] + estoque_venda_p3.values.tolist())

In [ ]:
# Exportação 11
#estoque_venda_p4.fillna('', inplace=True)
#estoque_venda_p4_4.update([estoque_venda_p4.columns.values.tolist()] + estoque_venda_p4.values.tolist())

# **Final**

In [ ]:
# Recado Final
print("Dados exportados com sucesso para o Google Sheets!")